In [7]:
import time
import torch
import os

In [7]:
import argparse
def get_train_args():
    parser=argparse.ArgumentParser()
    parser.add_argument('--batch_size',type=int,default=10,help = '每批数据的数量')
    parser.add_argument('--nepoch',type=int,default=3,help = '训练的轮次')
    parser.add_argument('--lr',type=float,default=0.001,help = '学习率')
    parser.add_argument('--gpu',type=bool,default=True,help = '是否使用gpu')
    parser.add_argument('--num_workers',type=int,default=2,help='dataloader使用的线程数量')
    parser.add_argument('--num_labels',type=int,default=3,help='分类类数')
    parser.add_argument('--data_path',type=str,default='./data',help='数据路径')
    opt=parser.parse_args()
    print(opt)
    return opt

opt = get_train_args()

NameError: name 'argparse' is not defined

In [2]:
class DotDict(dict):
    def __init__(self, *args, **kwargs):
        dict.__init__(self, *args, **kwargs)
        self.__dict__ = self

d = DotDict()
d.foo = 'bar'
d.foo1 = 'bar1'
print(d['foo'])
print(d.foo)

for k,v in d.items():
    print(k,v)

opt1 = DotDict({'num_labels' : 3})
print(opt1.num_labels)

opt2 = DotDict(num_labels = 3,)
print(opt2.num_labels)

bar
bar
foo bar
foo1 bar1
3
3


In [12]:
from transformers import *
import logging
logging.basicConfig(level=logging.INFO)

opt3 = DotDict(
    num_labels = 3,
    batch_size = 10,
    num_workers = 8,
    cache_dir = '../../PretrainedData/Transformers/bert-base-chinese',
    data_path = 'data',
    nepoch = 3,
               )

In [4]:
# num_labels是分类的类数
model = BertForSequenceClassification.from_pretrained('bert-base-chinese',
                                                         num_labels=opt3.num_labels,
                                                         cache_dir = opt3.cache_dir,
                                                         )
tokenizer = BertTokenizer.from_pretrained('bert-base-chinese',
                                          cache_dir = opt3.cache_dir,)
print(tokenizer)

INFO:transformers.configuration_utils:loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-chinese-config.json from cache at ../../PretrainedData/Transformers/bert-base-chinese\8a3b1cfe5da58286e12a0f5d7d182b8d6eca88c08e26c332ee3817548cf7e60a.3767c74c8ed285531d04153fe84a0791672aff52f7249b27df341dbce09b8305
INFO:transformers.configuration_utils:Model config BertConfig {
  "_num_labels": 3,
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bad_words_ids": null,
  "bos_token_id": null,
  "decoder_start_token_id": null,
  "directionality": "bidi",
  "do_sample": false,
  "early_stopping": false,
  "eos_token_id": null,
  "finetuning_task": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "is_decoder": false,
  "is_encoder_decoder": false,
  "labe

In [5]:
class NewsData(torch.utils.data.Dataset):
    def __init__(self, root, is_train=1):
        self.data_num = 7346
        self.x_list = []
        self.y_list = []
        self.posi = []

        with open(root + '/Train_DataSet.csv', encoding='UTF-8') as f:
            for i in range(self.data_num + 1):
                line = f.readline()[:-1] + '这是一个中性的数据'

                data_one_str = line.split(',')[len(line.split(',')) - 2]
                data_two_str = line.split(',')[len(line.split(',')) - 1]

                if len(data_one_str) < 6:
                    z = len(data_one_str)
                    data_one_str = data_one_str + '，' + data_two_str[0:min(200, len(data_two_str))]
                else:
                    data_one_str = data_one_str
                if i == 0:
                    continue

                word_l = tokenizer.encode(data_one_str, add_special_tokens=False)
                if len(word_l) < 100:
                    while (len(word_l) != 100):
                        word_l.append(0)
                else:
                    word_l = word_l[0:100]

                word_l.append(102)
                l = word_l
                word_l = [101]
                word_l.extend(l)

                self.x_list.append(torch.tensor(word_l))

                self.posi.append(torch.tensor([i for i in range(102)]))

        with open(root + '/Train_DataSet_Label.csv', encoding='UTF-8') as f:
            for i in range(self.data_num + 1):
                tempStr = f.readline()
                if i % 500 == 499:
                    print(i + 1, tempStr)
                label_one = tempStr[-2]
                if i == 0:
                    continue
                label_one = int(label_one)
                self.y_list.append(torch.tensor(label_one))

        # 训练集或者是测试集
        if is_train == 1:
            self.x_list = self.x_list[0:6000]
            self.y_list = self.y_list[0:6000]
            self.posi = self.posi[0:6000]
        else:
            self.x_list = self.x_list[6000:]
            self.y_list = self.y_list[6000:]
            self.posi = self.posi[6000:]

        self.len = len(self.x_list)

    def __getitem__(self, index):
        return self.x_list[index], self.y_list[index], self.posi[index]

    def __len__(self):
        return self.len

def get_data(opt):
    #NewsData继承于pytorch的Dataset类
    trainset = NewsData(opt.data_path,is_train = 1)
    trainloader=torch.utils.data.DataLoader(trainset,
                                            batch_size=opt.batch_size,
                                            shuffle=True,
                                            num_workers=opt.num_workers)
    testset = NewsData(opt.data_path,is_train = 0)
    testloader=torch.utils.data.DataLoader(testset,
                                           batch_size=opt.batch_size,
                                           shuffle=False,
                                           num_workers=opt.num_workers)
    return trainloader, testloader
trainloader, testloader = get_data(opt3)

500 69782363f8264714b60ba699aff87e61,2

1000 d6b520b3a93643c7855cc9ca4262bd0f,1

1500 4c4990a462b24b2787ddb75d69d68beb,1

2000 c67b1254e8a746a78df818c54001ee8d,1

2500 45e0fe90c3e845c397e8e7c8a021019d,2

3000 80aea805d1064025864224914ae28949,1

3500 77486e8fafa94f42985c20a46c0b5b0a,2

4000 21e36b376b2e4b4988736494c995e57f,1

4500 728cbcbcecf84cfe87ded2b2f2801616,2

5000 8ad5c7afb24948b0b3d1aae7ec6311c0,1

5500 4e3c9fa942d9425b90397f4a0b008180,1

6000 c7d498d2bac14644b7f8cfb67a3fb3ed,2

6500 35f7358beb72448095ffe1579128d1d4,2

7000 86486e5c075c4534ab7a0e2c3240108c,2

500 69782363f8264714b60ba699aff87e61,2

1000 d6b520b3a93643c7855cc9ca4262bd0f,1

1500 4c4990a462b24b2787ddb75d69d68beb,1

2000 c67b1254e8a746a78df818c54001ee8d,1

2500 45e0fe90c3e845c397e8e7c8a021019d,2

3000 80aea805d1064025864224914ae28949,1

3500 77486e8fafa94f42985c20a46c0b5b0a,2

4000 21e36b376b2e4b4988736494c995e57f,1

4500 728cbcbcecf84cfe87ded2b2f2801616,2

5000 8ad5c7afb24948b0b3d1aae7ec6311c0,1

5500 4e3c9fa942d94

In [8]:
print('cuda.is_available',torch.cuda.is_available())

if torch.cuda.is_available():
    model.cuda()

cuda.is_available True


In [9]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [10]:
def train(epoch, model, trainloader, testloader, optimizer, opt):
    print('\ntrain-Epoch: %d' % (epoch + 1))
    model.train()
    start_time = time.time()
    print_step = int(len(trainloader) / 10)
    for batch_idx, (sue, label, posi) in enumerate(trainloader):
        if opt.gpu:
            sue = sue.cuda()
            posi = posi.cuda()
            label = label.unsqueeze(1).cuda()

        optimizer.zero_grad()
        # 输入参数为词列表、位置列表、标签
        outputs = model(sue, position_ids=posi, labels=label)

        loss, logits = outputs[0], outputs[1]
        loss.backward()
        optimizer.step()

        if batch_idx % print_step == 0:
            print("Epoch:%d [%d|%d] loss:%f" % (epoch + 1, batch_idx, len(trainloader), loss.mean()))
    print("time:%.3f" % (time.time() - start_time))

In [13]:
def testFunc(epoch, model, trainloader, testloader, opt):
    print('\ntest-Epoch: %d' % (epoch + 1))
    model.eval()
    total = 0
    correct = 0
    with torch.no_grad():
        for batch_idx, (sue, label, posi) in enumerate(testloader):
            if opt.gpu:
                sue = sue.cuda()
                posi = posi.cuda()
                labels = label.unsqueeze(1).cuda()
                label = label.cuda()
            else:
                labels = label.unsqueeze(1)

            outputs = model(sue, labels=labels)
            loss, logits = outputs[:2]
            _, predicted = torch.max(logits.data, 1)

            total += sue.size(0)
            correct += predicted.data.eq(label.data).cpu().sum()

    s = ("Acc:%.3f" % ((1.0 * correct.numpy()) / total))
    print(s)

if not os.path.exists('./model.pth'):
    for epoch in range(opt3.nepoch):
        train(epoch, model, trainloader, testloader, optimizer, opt3)
        testFunc(epoch, model, trainloader, testloader, opt3)
    torch.save(model.state_dict(), './model.pth')
else:
    model.load_state_dict(torch.load('model.pth'))
    print('模型存在,直接test')
    testFunc(0, model, trainloader, testloader, opt3)


train-Epoch: 1


BrokenPipeError: [Errno 32] Broken pipe